In [2]:
import xspec as x
import matplotlib.pyplot as plt
import os
import pandas as pd
import numpy as np
from astropy.io import fits



In [3]:

#Load relxill and test it

# --- STEP 1: DEFINE PATHS ---
# Path to the folder where you compiled relxill (contains lmodel.dat)
relxill_dir = "/home/kyle/software/relxill"

# Path to the folder containing the heavy FITS tables (often the same dir)
relxill_tables = "/home/kyle/software/relxill"

# --- STEP 2: SET ENVIRONMENT VARIABLE ---
# This tells relxill where to look for the tables so you don't have to copy them
os.environ["RELXILL_TABLE_PATH"] = relxill_tables

# --- STEP 3: LOAD THE MODEL ---
# This mimics the "lmod relxill /path" command
print("Loading Relxill...")
x.AllModels.lmod("relxill", relxill_dir)

# --- STEP 4: USE IT ---
# Now you can use 'relxill', 'relxillCp', 'relxilllp', etc.
# Example: simple relativistic reflection
m = x.Model("tbabs * relxillns")

# Verify it loaded
print(m.componentNames)

Loading Relxill...
Model package relxill successfully loaded.
['TBabs', 'relxillNS']
tbvabs Version 2.3
Cosmic absorption with grains and H2, modified from
Wilms, Allen, & McCray, 2000, ApJ 542, 914-924
Questions: Joern Wilms
joern.wilms@sternwarte.uni-erlangen.de
joern.wilms@fau.de

http://pulsar.sternwarte.uni-erlangen.de/wilms/research/tbabs/

PLEASE NOTICE:
To get the model described by the above paper
you will also have to set the abundances:
   abund wilm

Note that this routine ignores the current cross section setting
as it always HAS to use the Verner cross sections as a baseline.
 *** loading RELXILL model (version 2.5) *** 

Model TBabs<1>*relxillNS<2> Source No.: 1   Active/Off
Model Model Component  Parameter  Unit     Value
 par  comp
   1    1   TBabs      nH         10^22    1.00000      +/-  0.0          
   2    2   relxillNS  Index1              3.00000      frozen
   3    2   relxillNS  Index2              3.00000      frozen
   4    2   relxillNS  Rbr              

In [4]:
# ... existing code ...
# --- 1. SETUP & DATA LOADING ---
# 1. Clear any existing data/models
x.AllData.clear()
x.AllModels.clear()

# 2. Define your file paths
pn_spec = "../products/0865600201/pn/spec/pn_source_spectrum.fits"
pn_bkg  = "../products/0865600201/pn/spec/pn_bkg_spectrum.fits"
pn_rmf  = "../products/0865600201/pn/spec/pn_rmf.rmf"
pn_arf  = "../products/0865600201/pn/spec/pn_arf.arf"

# 3. Load the PN spectrum
# This assigns the file to the first data group and first spectrum slot
x.AllData(pn_spec)

# 4. Assign Background and Response (RMF + ARF)
s = x.AllData(1)
s.background = pn_bkg
s.response = pn_rmf
s.response.arf = pn_arf

# Add this after assigning responses
x.AllData.ignore("bad")
s.ignore("**-0.8 12.0-**")
# s.ignore("7.5-9.0")

# Set Global XSPEC Settings for PN (usually Energy/keV)
x.Plot.device = "/null"
x.Plot.xAxis = "keV"
x.Xset.xsect = "vern"
x.Xset.abund = "wilm"


# --- 2. MODEL DEFINITION ---
# Defining the model for the loaded PN data
m1 = x.Model("tbabs * (nthcomp + diskbb)")



1 spectrum  in use
 
Spectral Data File: ../products/0865600201/pn/spec/pn_source_spectrum.fits  Spectrum 1
Net count rate (cts/s) for Spectrum:1  4.220e+00 +/- 7.206e-03
 Assigned to Data Group 1 and Plot Group 1
  Noticed Channels:  1-4096
  Telescope: XMM Instrument: EPN  Channel Type: PI
  Exposure Time: 8.128e+04 sec
 Using fit statistic: chi
 No response loaded.

***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Net count rate (cts/s) for Spectrum:1  5.948e-01 +/- 2.673e-02 (14.1 % total)
***Warning!  One or more spectra are missing responses,
               and are not suitable for fit.
Response successfully loaded.
Arf successfully loaded.

ignore:     0 channels ignored from  source number 1
   155 channels (1-155) ignored in spectrum #     1
  1697 channels (2400-4096) ignored in spectrum #     1

 Cross Section Table set to vern:  Verner, Ferland, Korista, and Yakovlev 1996
 Solar Abundance Vector set to wilm:  Wilms, J., A

In [5]:
# --- 4. FITTING ---



m1(1).values = 0.3
m1(1).frozen = False
m1(2).values = 1.7
m1(3).values = 100
m1(4).link = "p8"

m1(5).values = 1
m1(5).frozen = True
m1(9).values = 1




Fit statistic  : Chi-Squared              2.096983e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1

Test statistic : Chi-Squared              2.096983e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number(s): 1 

 Null hypothesis probability of 0.000000e+00 with 2238 degrees of freedom
 Current data and model not fit yet.
  parameter 1 is not frozen.

Fit statistic  : Chi-Squared              2.096983e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1

Test statistic : Chi-Squared              2.096983e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number(s): 1 

 Null hypothesis probability of 0.000000e+00 with 2238 degrees of freedom
 Current data and model not fit yet.

Fit st

In [6]:
x.Fit.statMethod = "chi"
x.Fit.query = "yes"
x.Fit.perform()

Default fit statistic is set to: Chi-Squared
   This will apply to all current and newly loaded spectra.

Fit statistic  : Chi-Squared              5.506664e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number: 1

Test statistic : Chi-Squared              5.506664e+09     using 2244 bins.

***Warning: Chi-square may not be valid due to bins with zero variance
            in spectrum number(s): 1 

 Null hypothesis probability of 0.000000e+00 with 2238 degrees of freedom
 Current data and model not fit yet.
                                   Parameters
Chi-Squared  |beta|/N    Lvl          1:nH       2:Gamma        3:kT_e        7:norm         8:Tin        9:norm
2782.11      263.97       -1      0.250847       1.67274       17.5481    0.00334680      0.931399     0.0173632
2762.6       23364.5      -1      0.251498       1.65673       12.8295    0.00332111      0.887372     0.0383587
2762.36      1910.44      -2

In [7]:
# --- NEATLY PRINT BEST FIT OUTPUT ---
print("\n" + "="*60)
print(f"{'COMPONENT':<15} | {'PARAMETER':<15} | {'VALUE':<12} | {'ERROR'}")
print("-" * 60)

# Iterate through all components in the first model
for comp_name in m1.componentNames:
    comp = getattr(m1, comp_name)
    for par_name in comp.parameterNames:
        par = getattr(comp, par_name)
        
        # Determine if it's frozen or has an error calculated
        err_str = "FROZEN" if par.frozen else f"+/- {par.sigma:.5f}"
        if par.link != "":
            err_str = f"LINKED ({par.link})"
            
        print(f"{comp_name:<15} | {par_name:<15} | {par.values[0]:<12.5f} | {err_str}")

print("-" * 60)
print(f"Final {x.Fit.statMethod.upper()}: {x.Fit.statistic:.2f}")
print(f"Degrees of Freedom: {x.Fit.dof}")
print(f"Reduced Statistic: {x.Fit.statistic/x.Fit.dof:.3f}")
print("="*60 + "\n")



COMPONENT       | PARAMETER       | VALUE        | ERROR
------------------------------------------------------------
TBabs           | nH              | 0.22399      | +/- 0.01749
nthComp         | Gamma           | 1.01371      | +/- 0.13989
nthComp         | kT_e            | 34.40633     | +/- 245.26853
nthComp         | kT_bb           | 1.43999      | LINKED (= p8)
nthComp         | inp_type        | 1.00000      | FROZEN
nthComp         | Redshift        | 0.00000      | FROZEN
nthComp         | norm            | 0.00077      | +/- 0.00070
diskbb          | Tin             | 1.43999      | +/- 0.19460
diskbb          | norm            | 0.20430      | +/- 0.03320
------------------------------------------------------------
Final CHI: 2662.86
Degrees of Freedom: 2238
Reduced Statistic: 1.190



In [8]:
# --- 5. PLOTTING ---

# Set rebinning for the plot only (e.g., min significance 7, max 10 bins)
x.Plot.setRebin(3, 10)
x.Plot.device = "/null"
x.Plot.xAxis = "keV"

# Extract Data
x.Plot("data")
energy_x   = x.Plot.x(1)
energy_err = x.Plot.xErr(1)
data_y     = x.Plot.y(1)
data_err   = x.Plot.yErr(1)
model_y    = x.Plot.model(1)

# Extract Residuals
x.Plot("delchi")
residuals  = x.Plot.y(1)

# Convert to numpy arrays - using raw data without masking zero-count bins
e  = np.array(energy_x)
ee = np.array(energy_err)
dy = np.array(data_y)
de = np.array(data_err)
my = np.array(model_y)
res = np.array(residuals)

#background

x.Plot("background")
bkg = x.Plot.y(1)

In [9]:
# Plotting

%matplotlib notebook
plt.style.use('default')
plt.figure(figsize=(10, 8), dpi=100)
gs = plt.GridSpec(2, 1, height_ratios=[3, 1], hspace=0)

# Top panel: Data and Model
ax1 = plt.subplot(gs[0])
ax1.errorbar(e, dy, xerr=ee, yerr=de, fmt='o', ms=2, elinewidth=0.8, label='PN Data')
ax1.step(e, my, where='mid', color='C1', lw=1.5, zorder=10, label='Model')

ax1.set_yscale("log")
ax1.set_ylabel(r"Counts s$^{-1}$ keV$^{-1}$")
ax1.set_title(f"PN Spectrum: {m1.expression}")
ax1.legend()
ax1.set_xticklabels([]) # Hide x-labels for top plot

ax1.plot(e, bkg, color='black', lw=1.0, alpha=0.5, label='Background')

# Bottom panel: Residuals
ax2 = plt.subplot(gs[1])
ax2.step(e, res, where='mid', color='black', lw=1.2)
# ax2.errorbar(e, res, xerr=ee, fmt='none', ecolor='black', elinewidth=0.8, capsize=2)
ax2.axhline(0, color='red', linestyle='--')
# ax2.set_ylim(-4,4)
ax2.set_xlabel("Energy (keV)")
ax2.set_ylabel(r"Residuals (\$\chi\$)")

plt.tight_layout()



<IPython.core.display.Javascript object>

In [10]:
# # Quick check: Calculate counts from rates to decide on statistic
# exposure = s.exposure
# counts = np.array([r * exposure for r in s.values if r > 0])
# min_counts = np.min(counts)
# median_counts = np.median(counts)
#
# print(f"Exposure: {exposure:.1f} s")
# print(f"Min counts/bin: {min_counts:.1f}")
# print(f"Median counts/bin: {median_counts:.1f}")
#
# if min_counts < 20:
#     print("\n--- RECOMMENDATION ---")
#     print(f"Minimum counts ({min_counts:.1f}) is below 20.")
#     print("Gaussian statistics (Chi-Squared) are invalid for low-count bins.")
#     print("Use 'cstat' for the most reliable results.")
# else:
#     print("\n--- RECOMMENDATION ---")
#     print(f"All bins have >= 20 counts (Min: {min_counts:.1f}).")
#     print("You can safely use 'chi' (Chi-Squared) or 'cstat'.")